# Parametrizing Value.NativeQuery For Query Folding
> Passing parameters and lists to Value.NativeQuery to make queries foldable and dynamic

- toc: true 
- badges: true
- comments: true
- categories: [Power BI, powerquery, queryfolding, M, optimization]
- hide: false

## Query Folding

In a recent [video by Guy In A Cube](https://www.youtube.com/watch?v=lopTsclrpHI), Adam showed a great trick to pass a list to the `where` clause in a SQL statement. I want to show another approach which is more parametric and can fold. It does come with one drawback that I will share. Watch Adam's video first:

>youtube:https://youtu.be/lopTsclrpHI

#### Query Folding Using Native.Query()

It's known that instead of pasting query in the SQL server connection dialog box like below, you can use the same SQL statement in `Value.NativeQuery()` to make it foldable. This has the advantage of potentially speeding up the refresh times by offloading all the processing to the server instead of the gateway. I highly recommend reading [this blog](https://blog.crossjoin.co.uk/2021/02/21/query-folding-on-sql-queries-in-power-query-using-value-nativequery-and-enablefoldingtrue/) by Chris Webb, which is how I learned about this function. I have also added few more resources about Query Folding at the end of the blog, please check them out.   

This will not fold :

![1](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq1.png)

##### Using Native.Query()

This will fold:

![2](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq2.png)

What's not generally known is that you can pass parameters to the `Value.NativeQuery()` using the optional third argument. Again, I learned about it from Chris' blog [here](https://blog.crossjoin.co.uk/2016/12/11/passing-parameters-to-sql-queries-with-value-nativequery-in-power-query-and-power-bi/). I modified the SQL statement and passed a parameter `@Date` by using the third optional argument as shown below. You can use as many parameters as you like. The query is still foldable.  

>note: Be sure to set [EnableFolding=true] to fold subsequent query steps. If you don't specify this, your initial query will fold but not the queries following the initial query

![3](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq3.png)

##### But What About Lists ?

Good question. If we try to pass a list in a `where` clause like Adam did, however, it will not work. See below. The parameters only work for single values.

![4](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq4.png)

![5](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq5.png)

## The Solution : Use JSON

The solution to this is using `OPENJSON` format to pass the values. I have not tested it against all the foldable sources. It certainly works with SQL Server and Synapse Analytics. I found this solution last year on a forum somewhere and I can't find the source anymore unfortunately. If anyone knows it, please let me know so I can attribute to the original author.

Create the parameter as a JSON object and read it in SQl statement using OPENJSON

![6](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq6.png)

![7](https://raw.githubusercontent.com/pawarbi/blog/master/images/nq7.png)

##### What If You Have An Existing List ?

If you have an existing list that's derived from a column, you can use Text.Combine(), like Adam did, and pass it to the JSON parameter we created. If you want to pass numbers to the `where` clause like I did above, here are the steps to pass your list named `_mylist` as a parameter:


- First convert the numbers to text, you can use `Number.ToText()`. Easier to just convert the column to text before creating the list
- Concatenate the list: `"[" & Text.Combine(_mylist,",") & "]"`
- Now you can specify this list as a parameter directly in the `Value.NativeQuery()` parameters option

The only drawback here is the automatically generated SQL query does not show the values passed to `IN`. But it folds, that's what matters.

## Resources

Other than [Chris Webb's blog](https://blog.crossjoin.co.uk/), here are couple more resources on Query Folding:

##### 30DQUERY Challenge

If you want to improve your M and learn the techniques to make queries foldable, I highly recommend participating in Alex Powers' #30DQUERY challenge.

>youtube: https://youtu.be/9sV3hIn8VTY

##### Ben Gribaudo's Blog

[Ben's blog](https://bengribaudo.com/blog/2022/01/20/6500/power-query-m-primer-part-23-query-folding-i) is the most comprehensive resource on Query Folding. If you want to get deeper understanding of query folding and M in general, you have to read Ben's blog. 

Watch [**How Power Query Thinks**](https://bengribaudo.com/blog/2020/08/26/5417/how-power-query-thinks) by Ben